# Hands Signs Digits Classification Using Resnets

In [2]:
# import packages
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import h5py

%matplotlib inline
np.random.seed(1)
tf.random.set_seed(2)

In [3]:
# load and prepare data
train_dataset = h5py.File('data/train_signs.h5', 'r')
X_train_orig = np.array(train_dataset['train_set_x'][:])
Y_train_orig = np.array(train_dataset['train_set_y'][:])

test_dataset = h5py.File('data/test_signs.h5', 'r')
X_test_orig = np.array(test_dataset['test_set_x'][:])
Y_test_orig = np.array(test_dataset['test_set_y'][:])

classes = np.array(test_dataset['list_classes'][:])

Y_train_orig = Y_train_orig.reshape((1, Y_train_orig.shape[0]))
Y_test_orig = Y_test_orig.reshape((1, Y_test_orig.shape[0]))

X_train = X_train_orig / 255.
X_test = X_test_orig / 255.
print(X_train.shape)
print(X_test.shape)

Y_train = np.eye(6)[Y_train_orig.reshape(-1)]
Y_test = np.eye(6)[Y_test_orig.reshape(-1)]
print(Y_train.shape)
print(Y_test.shape)

(1080, 64, 64, 3)
(120, 64, 64, 3)
(1080, 6)
(120, 6)


In [ ]:
# utility functions for residual blocks

def identity_block(X, f, filters):
    '''
    This block is used when the input activation has the same dimension as the output activation.
    The shortcut path skips over 3 layers.
    '''
    F1, F2, F3 = filters # number of filters for each layer
    X_shortcut = X # save the input value for the shortcut path
    
    # first layer
    X = layers.Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Activation('relu')(X)

    # second layer
    X = layers.Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same')(X)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Activation('relu')(X)

    # third layer
    X = layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = layers.BatchNormalization(axis=3)(X)

    # add main path with shortcut path and apply activation
    X = layers.Add()([X, X_shortcut])
    X = layers.Activation('relu')(X)

    return X

def convolution_block(X, f, filters, s=2):
    '''
    This block is used when the input activation doesn't have the same dimension as the output activation.
    '''
    F1, F2, F3 = filters # number of filters for each layer
    X_shortcut = X # save the input value for the shortcut path
    
    # first layer
    X = layers.Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid')(X)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Activation('relu')(X)

    # second layer
    X = layers.Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same')(X)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Activation('relu')(X)

    # third layer
    X = layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = layers.BatchNormalization(axis=3)(X)

    # shortcut path
    X_shortcut = layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid')(X_shortcut)
    X_shortcut = layers.BatchNormalization(axis=3)(X_shortcut)

    # add main path with shortcut path and apply activation
    X = layers.Add()([X, X_shortcut])
    X = layers.Activation('relu')(X)

    return X

In [ ]:
# build the ResNet50 model
